In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_excel('Insurancefraud.xlsx', sheet_name='Fraud')
df.head()

,months_as_customer,age,policy_number,policy_initial_date,policy_state,limit/person,limit_accident,policy_deductable,policy_annual_premium,umbrella_limit,...,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250,500,1000,1406.91,0,...,YES,1,2,YES,71610,6510,13020,52080,2004,1
1,228,42,342868,2006-06-27,IN,250,500,2000,1197.22,5000000,...,NO,0,0,NO,5070,780,780,3510,2007,1
2,134,29,687698,2000-09-06,OH,100,300,2000,1413.14,5000000,...,NO,2,3,NO,34650,7700,3850,23100,2007,0
3,256,41,227811,1990-05-25,IL,250,500,2000,1415.74,6000000,...,NO,1,2,NO,63400,6340,6340,50720,2014,1
4,228,44,367455,2014-06-06,IL,500,1000,1000,1583.91,6000000,...,NO,0,1,NO,6500,1300,650,4550,2009,0


In [3]:
wacc = 0.0764
np.random.seed(12345)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df1 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df1




,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.492188,0.507812,0.904762,0.095238,1020.534982
1,RIDGE,0.492188,0.507812,0.904762,0.095238,1020.534982
2,LOGIT,0.445312,0.554688,0.902116,0.097884,623.976677
3,LASSO LOGIT,0.460938,0.539062,0.902116,0.097884,746.707927
4,RANDOM FOREST,0.265625,0.734375,0.944444,0.055556,-333.599809


In [4]:
f1 = y_test.sum()

print(f1)

128


In [5]:
c1 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p1 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c1)
print(p1)

61365.625
1255.0343750000002


In [6]:
wacc = 0.0764
np.random.seed(56455)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df2 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df2




,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.425197,0.574803,0.892761,0.107239,427.686446
1,RIDGE,0.425197,0.574803,0.898123,0.101877,485.026180
2,LOGIT,0.370079,0.629921,0.906166,0.093834,139.222317
3,LASSO LOGIT,0.385827,0.614173,0.900804,0.099196,205.257858
4,RANDOM FOREST,0.259843,0.740157,0.932976,0.067024,-437.705941


In [7]:
f2 = y_test.sum()

print(f2)

127


In [8]:
c2 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p2 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c2)
print(p2)

61687.637795275594
1222.1902362204723


In [9]:
wacc = 0.0764
np.random.seed(2335312)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df3 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df3




,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.491228,0.508772,0.905612,0.094388,895.224404
1,RIDGE,0.491228,0.508772,0.908163,0.091837,919.518514
2,LOGIT,0.429825,0.570175,0.903061,0.096939,442.532136
3,LASSO LOGIT,0.412281,0.587719,0.915816,0.084184,441.603212
4,RANDOM FOREST,0.368421,0.631579,0.913265,0.086735,111.310418


In [10]:
f3 = y_test.sum()

print(f3)

114


In [11]:
c3 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p3 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c3)
print(p3)

61199.73684210526
1280.9591228070178


In [12]:
wacc = 0.0764
np.random.seed(6545678)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df4 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df4




,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.507692,0.492308,0.865169,0.134831,603.219463
1,RIDGE,0.507692,0.492308,0.867978,0.132022,632.145271
2,LOGIT,0.415385,0.584615,0.884831,0.115169,109.329036
3,LASSO LOGIT,0.461538,0.538462,0.867978,0.132022,283.959732
4,RANDOM FOREST,0.246154,0.753846,0.918539,0.081461,-820.241588


In [13]:
f4 = y_test.sum()

print(f4)

130


In [14]:
c4 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p4 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c4)
print(p4)

58030.92307692308
1237.4935384615385


In [15]:
wacc = 0.0764
np.random.seed(987611234)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df5 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df5




,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.558333,0.441667,0.879357,0.120643,1212.071575
1,RIDGE,0.558333,0.441667,0.879357,0.120643,1212.071575
2,LOGIT,0.408333,0.591667,0.892761,0.107239,249.411226
3,LASSO LOGIT,0.450000,0.550000,0.882038,0.117962,446.972972
4,RANDOM FOREST,0.300000,0.700000,0.906166,0.093834,-408.692456


In [16]:
f5 = y_test.sum()

print(f5)

120


In [17]:
c5 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p5 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c5)
print(p5)

60911.333333333336
1232.8699166666668


In [18]:
wacc = 0.0764
np.random.seed(85137122)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df6 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df6




,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.385246,0.614754,0.920981,0.079019,371.259872
1,RIDGE,0.393443,0.606557,0.918256,0.081744,404.703582
2,LOGIT,0.352459,0.647541,0.923706,0.076294,154.140178
3,LASSO LOGIT,0.360656,0.639344,0.923706,0.076294,215.365506
4,RANDOM FOREST,0.180328,0.819672,0.942779,0.057221,-937.120380


In [19]:
f6 = y_test.sum()

print(f6)

122


In [20]:
c6 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p6 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c6)
print(p6)

61225.32786885246
1233.430163934426


In [21]:
wacc = 0.0764
np.random.seed(98765)
desired_probability = 0.2

df['Train']=[1 if np.random.uniform()<0.5 else 0 for _ in range(len(df))]
df.head()

x=df.drop(columns=['fraud_reported', 'Train'])
y=df['fraud_reported']

x_train = x[df['Train'] == 1]
#It selects the rows on x, where the conditiomn is true
x_test = x[df['Train'] == 0]
#It selects the rows on x, where the conditiomn is true
y_train = y[df['Train'] == 1]
#It selects the rows on y, where the conditiomn is true
y_test = y[df['Train'] == 0]
#It selects the rows on y, where the conditiomn is true

numeric_df = x_train.select_dtypes(include=['number'])
numeric_test_df = x_test.select_dtypes(include=['number'])

categorical_df = x_train.select_dtypes(include=['object'])
categorical_test_df = x_test.select_dtypes(include=['object'])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
scaler.fit(numeric_df)

# Transform numeric features
X_train_numeric_processed = scaler.transform(numeric_df)
X_test_numeric_processed = scaler.transform(numeric_test_df)

encoder= OneHotEncoder()
encoder.fit(categorical_df)

X_train_categorical_processed = encoder.transform(categorical_df).toarray()
X_test_categorical_processed = encoder.transform(categorical_test_df).toarray()

X_train_processed = np.concatenate((X_train_numeric_processed, X_train_categorical_processed), axis=1)
X_test_processed = np.concatenate((X_test_numeric_processed, X_test_categorical_processed), axis=1)

total_samples = len(y_train)
fraud_sample = sum(y_train)
non_fraud_sample = total_samples - fraud_sample
sample_fraud = fraud_sample/total_samples


from sklearn import linear_model
ols=linear_model.LinearRegression()
ols.fit(X_train_processed, y_train)

ols_predictions = ols.predict(X_test_processed).round(0)
ols_predictions

from sklearn.metrics import mean_squared_error
mse_ols=mean_squared_error(y_test, ols_predictions)
mse_ols

ols_ct=pd.crosstab(y_test, ols_predictions)
ols_ct

type1_ols=ols_ct.iloc[0,1]/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1])
type1_ols
type2_ols=ols_ct.iloc[1,0]/(ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
type2_ols
fraud_acc_ols = ols_ct.iloc[1,1] / (ols_ct.iloc[1,1]+ols_ct.iloc[1,0])
fraud_acc_ols
nfraud_acc_ols = ols_ct.iloc[0,0] / (ols_ct.iloc[0,1]+ols_ct.iloc[0,0])
nfraud_acc_ols

r_ols=(ols_ct.iloc[0,0]+ols_ct.iloc[1,1])/(ols_ct.iloc[0,0]+ols_ct.iloc[0,1]+ols_ct.iloc[1,0]+ols_ct.iloc[1,1])
r_ols

from sklearn import linear_model
ridge=linear_model.Ridge()
ridge.fit(X_train_processed, y_train)

ridge.fit(X_train_processed, y_train).coef_

ridge_predictions = ridge.predict(X_test_processed).round(0)
ridge_predictions
mse_ridge=mean_squared_error(y_test, ridge_predictions)
mse_ridge
ridge_ct=pd.crosstab(y_test, ridge_predictions)
ridge_ct

type1_ridge=ridge_ct.iloc[0,1]/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1])
type1_ridge

type2_ridge=ridge_ct.iloc[1,0]/(ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
type2_ridge
fraud_acc_ridge = ridge_ct.iloc[1,1] / (ridge_ct.iloc[1,1]+ridge_ct.iloc[1,0])
fraud_acc_ridge
nfraud_acc_ridge = ridge_ct.iloc[0,0] / (ridge_ct.iloc[0,1]+ridge_ct.iloc[0,0])
nfraud_acc_ridge
r_ridge=(ridge_ct.iloc[0,0]+ridge_ct.iloc[1,1])/(ridge_ct.iloc[0,0]+ridge_ct.iloc[0,1]+ridge_ct.iloc[1,0]+ridge_ct.iloc[1,1])
r_ridge


from sklearn import linear_model
logit=linear_model.LogisticRegression()
logit.fit(X_train_processed, y_train)
logit_predictions = logit.predict(X_test_processed)
logit_predictions

mse_logit=mean_squared_error(y_test, logit_predictions)
mse_logit

logit_ct=pd.crosstab(y_test, logit_predictions)
logit_ct

type1_logit=logit_ct.iloc[0,1]/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1])
type1_logit

type2_logit=logit_ct.iloc[1,0]/(logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
type2_logit

fraud_acc_logit = logit_ct.iloc[1,1] / (logit_ct.iloc[1,1]+logit_ct.iloc[1,0])
fraud_acc_logit

nfraud_acc_logit = logit_ct.iloc[0,0] / (logit_ct.iloc[0,1]+logit_ct.iloc[0,0])
nfraud_acc_logit

r_logit=(logit_ct.iloc[0,0]+logit_ct.iloc[1,1])/(logit_ct.iloc[0,0]+logit_ct.iloc[0,1]+logit_ct.iloc[1,0]+logit_ct.iloc[1,1])
r_logit

from sklearn import linear_model
lasso_logit=linear_model.LogisticRegression(penalty='l1', solver='liblinear')
lasso_logit.fit(X_train_processed, y_train)

lasso_logit_predictions = lasso_logit.predict(X_test_processed)
lasso_logit_predictions

mse_lasso_logit=mean_squared_error(y_test, lasso_logit_predictions)
mse_lasso_logit

lasso_logit_ct=pd.crosstab(y_test, lasso_logit_predictions)
lasso_logit_ct

type1_lasso_logit=lasso_logit_ct.iloc[0,1]/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1])
type1_lasso_logit

type2_lasso_logit=lasso_logit_ct.iloc[1,0]/(lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
type2_lasso_logit
fraud_acc_lasso_logit = lasso_logit_ct.iloc[1,1] / (lasso_logit_ct.iloc[1,1]+lasso_logit_ct.iloc[1,0])
fraud_acc_lasso_logit
nfraud_acc_lasso_logit = lasso_logit_ct.iloc[0,0] / (lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[0,0])
nfraud_acc_lasso_logit 

r_lasso_logit=(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[1,1])/(lasso_logit_ct.iloc[0,0]+lasso_logit_ct.iloc[0,1]+lasso_logit_ct.iloc[1,0]+lasso_logit_ct.iloc[1,1])
r_lasso_logit

from sklearn.ensemble import RandomForestClassifier

n_estimators = int(np.sqrt(X_train_processed.shape[1]))

# Initialize Random Forest classifier with the calculated number of estimators
rf = RandomForestClassifier(n_estimators=n_estimators)

# Train the classifier
rf.fit(X_train_processed, y_train)

rf_predictions=rf.predict(X_test_processed)
rf_predictions

mse_rf=mean_squared_error(y_test, rf_predictions)
mse_rf

rf_ct=pd.crosstab(y_test, rf_predictions)
rf_ct

type1_rf=rf_ct.iloc[0,1]/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1])
type1_rf

type2_rf=rf_ct.iloc[1,0]/(rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
type2_rf

fraud_acc_rf = rf_ct.iloc[1,1] / (rf_ct.iloc[1,1]+rf_ct.iloc[1,0])
fraud_acc_rf

nfraud_acc_rf = rf_ct.iloc[0,0] / (rf_ct.iloc[0,1]+rf_ct.iloc[0,0])
nfraud_acc_rf

r_rf=(rf_ct.iloc[0,0]+rf_ct.iloc[1,1])/(rf_ct.iloc[0,0]+rf_ct.iloc[0,1]+rf_ct.iloc[1,0]+rf_ct.iloc[1,1])
r_rf

mse_values=[mse_ols, mse_ridge, mse_logit, mse_lasso_logit, mse_rf]
pseudo_r2=[r_ols, r_ridge, r_logit,r_lasso_logit, r_rf]
type1=[type1_ols, type1_ridge, type1_logit, type1_lasso_logit, type1_rf]
type2=[type2_ols, type2_ridge, type2_logit, type2_lasso_logit, type2_rf]
fraud_acc=[fraud_acc_ols, fraud_acc_ridge, fraud_acc_logit, fraud_acc_lasso_logit, fraud_acc_rf]
nfraud_acc=[nfraud_acc_ols, nfraud_acc_ridge, nfraud_acc_logit, nfraud_acc_lasso_logit, nfraud_acc_rf]

total_claim = x_test.loc[y_test == 1, 'total_claim_amount'].sum()

total_policy =((x_test.loc[y_test ==1, 'policy_annual_premium'])/wacc).sum()

total_investigation_costs = y_test.count() * 550

legal_fees = 0.365

benefit = [(total_claim*fraud_acc_ols + total_investigation_costs - (1+legal_fees)*total_claim*type1_ols - total_policy)/10**3, 
          (total_claim*fraud_acc_ridge + total_investigation_costs - (1+legal_fees)*total_claim*type1_ridge - total_policy)/10**3,
          (total_claim*fraud_acc_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_logit - total_policy)/10**3,
          (total_claim*fraud_acc_lasso_logit + total_investigation_costs - (1+legal_fees)*total_claim*type1_lasso_logit - total_policy)/10**3, 
          (total_claim*fraud_acc_rf + total_investigation_costs - (1+legal_fees)*total_claim*type1_rf - total_policy)/10**3]

model_names=['OLS', 'RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
summary_df7 = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': fraud_acc, 
    'Type 2 error': type2, 
    'Not Fraud prediction accuracy': nfraud_acc,
    'Type 1 error': type1,
    'Benefit in thousands': benefit})

summary_df7

f7 = y_test.sum()

print(f7)

c7 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p7 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c7)
print(p7)

123
61124.06504065041
1257.2066666666665


In [22]:
f7 = y_test.sum()

print(f7)

123


In [23]:
c7 = x_test.loc[y_test == 1, 'total_claim_amount'].mean()
p7 = x_test.loc[y_test == 1, 'policy_annual_premium'].mean()
print(c7)
print(p7)

61124.06504065041
1257.2066666666665


In [24]:
summary_df1

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.492188,0.507812,0.904762,0.095238,1020.534982
1,RIDGE,0.492188,0.507812,0.904762,0.095238,1020.534982
2,LOGIT,0.445312,0.554688,0.902116,0.097884,623.976677
3,LASSO LOGIT,0.460938,0.539062,0.902116,0.097884,746.707927
4,RANDOM FOREST,0.265625,0.734375,0.944444,0.055556,-333.599809


In [25]:
summary_df2

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.425197,0.574803,0.892761,0.107239,427.686446
1,RIDGE,0.425197,0.574803,0.898123,0.101877,485.026180
2,LOGIT,0.370079,0.629921,0.906166,0.093834,139.222317
3,LASSO LOGIT,0.385827,0.614173,0.900804,0.099196,205.257858
4,RANDOM FOREST,0.259843,0.740157,0.932976,0.067024,-437.705941


In [26]:
summary_df3

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.491228,0.508772,0.905612,0.094388,895.224404
1,RIDGE,0.491228,0.508772,0.908163,0.091837,919.518514
2,LOGIT,0.429825,0.570175,0.903061,0.096939,442.532136
3,LASSO LOGIT,0.412281,0.587719,0.915816,0.084184,441.603212
4,RANDOM FOREST,0.368421,0.631579,0.913265,0.086735,111.310418


In [27]:
summary_df4

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.507692,0.492308,0.865169,0.134831,603.219463
1,RIDGE,0.507692,0.492308,0.867978,0.132022,632.145271
2,LOGIT,0.415385,0.584615,0.884831,0.115169,109.329036
3,LASSO LOGIT,0.461538,0.538462,0.867978,0.132022,283.959732
4,RANDOM FOREST,0.246154,0.753846,0.918539,0.081461,-820.241588


In [28]:
summary_df5

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.558333,0.441667,0.879357,0.120643,1212.071575
1,RIDGE,0.558333,0.441667,0.879357,0.120643,1212.071575
2,LOGIT,0.408333,0.591667,0.892761,0.107239,249.411226
3,LASSO LOGIT,0.450000,0.550000,0.882038,0.117962,446.972972
4,RANDOM FOREST,0.300000,0.700000,0.906166,0.093834,-408.692456


In [29]:
summary_df6

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.385246,0.614754,0.920981,0.079019,371.259872
1,RIDGE,0.393443,0.606557,0.918256,0.081744,404.703582
2,LOGIT,0.352459,0.647541,0.923706,0.076294,154.140178
3,LASSO LOGIT,0.360656,0.639344,0.923706,0.076294,215.365506
4,RANDOM FOREST,0.180328,0.819672,0.942779,0.057221,-937.120380


In [30]:
summary_df7

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.406504,0.593496,0.879896,0.120104,77.903559
1,RIDGE,0.406504,0.593496,0.879896,0.120104,77.903559
2,LOGIT,0.365854,0.634146,0.885117,0.114883,-174.127080
3,LASSO LOGIT,0.439024,0.560976,0.887728,0.112272,402.784349
4,RANDOM FOREST,0.170732,0.829268,0.950392,0.049608,-971.233564


In [31]:
dfs = [summary_df1, summary_df2, summary_df3, summary_df4, summary_df5, summary_df6, summary_df7]

In [32]:
len(dfs)

7

In [33]:
Average_Fraud_prediction_accuracy = (summary_df1['Fraud prediction accuracy']+ summary_df2['Fraud prediction accuracy'] + summary_df3['Fraud prediction accuracy'] + summary_df4['Fraud prediction accuracy'] + summary_df5['Fraud prediction accuracy'] + summary_df6['Fraud prediction accuracy']+ summary_df7['Fraud prediction accuracy'])/len(dfs)
Average_Fraud_prediction_accuracy

0    0.466627
1    0.467798
2    0.398178
3    0.424323
4    0.255872
Name: Fraud prediction accuracy, dtype: float64

In [34]:
Average_type2 = (summary_df1['Type 2 error']+ summary_df2['Type 2 error'] + summary_df3['Type 2 error'] + summary_df4['Type 2 error'] + summary_df5['Type 2 error'] + summary_df6['Type 2 error']+ summary_df7['Type 2 error'])/len(dfs)
Average_type2

0    0.533373
1    0.532202
2    0.601822
3    0.575677
4    0.744128
Name: Type 2 error, dtype: float64

In [35]:
Average_NFraud_prediction_accuracy = (summary_df1['Not Fraud prediction accuracy']+ summary_df2['Not Fraud prediction accuracy'] + summary_df3['Not Fraud prediction accuracy'] + summary_df4['Not Fraud prediction accuracy'] + summary_df5['Not Fraud prediction accuracy'] + summary_df6['Not Fraud prediction accuracy'] + summary_df7['Not Fraud prediction accuracy'])/len(dfs)
Average_NFraud_prediction_accuracy

0    0.892648
1    0.893791
2    0.899680
3    0.897169
4    0.929795
Name: Not Fraud prediction accuracy, dtype: float64

In [36]:
Average_type1 = (summary_df1['Type 1 error']+ summary_df2['Type 1 error'] + summary_df3['Type 1 error'] + summary_df4['Type 1 error'] + summary_df5['Type 1 error'] + summary_df6['Type 1 error']+ summary_df7['Not Fraud prediction accuracy'])/len(dfs)
Average_type1

0    0.215893
1    0.214751
2    0.210354
3    0.213610
4    0.198889
dtype: float64

In [37]:
Average_benefit = (summary_df1['Benefit in thousands'] + summary_df2['Benefit in thousands']+ summary_df3['Benefit in thousands']+ summary_df4['Benefit in thousands']+ summary_df5['Benefit in thousands']+ summary_df6['Benefit in thousands']+ summary_df7['Benefit in thousands'])/len(dfs)
Average_benefit

0    658.271472
1    678.843380
2    220.640642
3    391.807365
4   -542.469046
Name: Benefit in thousands, dtype: float64

El MSE es una medida del promedio de los cuadrados de los errores, es decir, la diferencia promedio al cuadrado entre los valores observados y los valores predichos por el modelo. Cuanto menor sea el MSE, mejor será el modelo en términos de precisión.

El Pseudo R-cuadrado es una medida que intenta proporcionar una interpretación similar al R-cuadrado para modelos que no son de mínimos cuadrados ordinarios, como la regresión logística. No se interpreta exactamente de la misma manera que el R-cuadrado en la regresión lineal, pero un valor más alto generalmente indica un mejor ajuste del modelo.

Error de Tipo 1 (Falsos Positivos): Un alto error de Tipo 1 significa que el modelo está identificando incorrectamente transacciones legítimas como fraudulentas. Esto puede ser costoso y molesto para los clientes legítimos y puede dañar la relación con los clientes y la reputación de la compañía de seguros. Predice que hay fraude, cuando no hay fraude.

El error de Tipo 2 se refiere a la tasa de falsos negativos, es decir, cuántas veces un modelo no logra detectar un evento (como el fraude) cuando en realidad sí ocurrió. Esto puede resultar en pérdidas financieras significativas para la compañía de seguros si los reclamos fraudulentos no se detectan. Predice que no hay fraude, cuando hay fraude

Number of False negatives: The number of instances where the model incorrectly predicts not fraud (negative class)

Number of actual Positives: Total number of instances where true outcome is fraud

Type II Error Rate= Number of False Negatives/Number of Actual Positives

Number of False Positives: The number of instances where the model incorrectly predicts default (positive class) when the true outcome is not default (negative class).

Number of Actual Negatives: The total number of instances where the true outcome is not default (negative class).

Type I Error Rate= Number of False Positives/Number of Actual Negatives

En el contexto de la prevención de fraudes en seguros, minimizar los errores de Tipo 2 generalmente sería la prioridad porque los reclamos fraudulentos pueden tener un impacto financiero significativo. Sin embargo, un modelo con un error de Tipo 1 extremadamente alto también podría ser impráctico porque sobrecargaría al equipo de revisión de reclamos con falsas alarmas.En el contexto de la prevención de fraudes en seguros, minimizar los errores de Tipo 2 generalmente sería la prioridad porque los reclamos fraudulentos pueden tener un impacto financiero significativo. Sin embargo, un modelo con un error de Tipo 1 extremadamente alto también podría ser impráctico porque sobrecargaría al equipo de revisión de reclamos con falsas alarmas.

## In $ terms

**If I predict fraud, I won´t pay anything.**

If correctly predict fraud, I will gain the "Total claim amount"
(I didn´t pay when I shouldn´t have)

If incorrectly predict fraud (type 1), I will lose a customer => lose policy annual premium
(I didn´t pay when I should have)

**If I predict not fraud, I will pay.**

If I correctly predict not fraud, I will "not lose" (gain) the policy annual premium
(I paid when I should have)

If I incorrectly predict not fraud (type 2), I will lose the "Total claim amount" (because I shouldn´t be paying it)
(I paid when I shouldn´t have)




## Benefit

If the prediction is "not fraud" the outcome would be the same as if the model had never been implemented. If the prediction is "Not fraud", no additional measure or actions would be implemented and the results wouln´t change. No incremental actions, measures, benefits or cashflows.

If the prediction is "fraud", there will be new actions: we wouldn´t pay. However, there will also be consequences. When we correctly predict fraud we won´t pay when we shouldn´t, so the money we won´t spend would be a gain (equal to the claim amount).  When we incorrectly predict fraud, we won´t pay when we should have. This will probably mean that we lose a customer and, therefore, lose the annual premium. 

The benefit of using this model is defined as Gain - Loss, where:

Gain:   Total claim when fraud occurs * probability of accurately predicting fraud 

Loss:   Total annual premium when fraud occurs * probability of incorrectly predicting fraud (type 1)


In [43]:
model_names=['OLS','RIDGE', 'LOGIT', 'LASSO LOGIT', 'RANDOM FOREST']
cv = pd.DataFrame({
    'Model': model_names,
    'Fraud prediction accuracy': Average_Fraud_prediction_accuracy, 
    'Type 2 error': Average_type2, 
    'Not Fraud prediction accuracy': Average_NFraud_prediction_accuracy,
    'Type 1 error': Average_type1,
    'Benefit in thousands': Average_benefit})
cv

,Model,Fraud prediction accuracy,Type 2 error,Not Fraud prediction accuracy,Type 1 error,Benefit in thousands
0,OLS,0.466627,0.533373,0.892648,0.215893,658.271472
1,RIDGE,0.467798,0.532202,0.893791,0.214751,678.843380
2,LOGIT,0.398178,0.601822,0.899680,0.210354,220.640642
3,LASSO LOGIT,0.424323,0.575677,0.897169,0.213610,391.807365
4,RANDOM FOREST,0.255872,0.744128,0.929795,0.198889,-542.469046


In [39]:
#Average fraud cases
n_fraud_average = (f1+f2+f3+f4+f5+f6+f7)/7
n_fraud_average

123.42857142857143

In [40]:
#Claim average
claim_average = (c1+c2+c3+c4+c5+c6+c7)/7
claim_average

60792.09270816288

In [41]:
#Premium average
premium_average = (p1+p2+p3+p4+p5+p6+p7)/7
premium_average

1245.5977171081124

A pesar de que LASSO LOGIT es el modelo con mayor error tipo 1, es el modelo que minimiza el error tipo 2, es decir, el modelo que mejor predice los casos de fraude.

Debido a que el error tipo 2 es "más caro" que el error tipo 2  (prima anual que la aseguradora recibe es significativamente menor a la cantidad de dinero que debe pagar en caso de siniestro), LASSO LOGIT es el modelo que maximiza los beneficios monetarios. 

En ese sentido, que la aseguradore use ese modelo le ayudará a ahorrar 218 mil de dólares por cada 122 casos de fraude, pues más del 40% de los fraudes son identificados.

Si extrapolamos estos resultados y aplicamos el modelo a más de 122 casos, las ganancias aumentarán significativamente.

In [42]:
f1

128